In [ ]:
%%time
# ? insert stat dump
import json
import cx_Oracle as cx
from flatten_json import flatten
from concurrent.futures import ThreadPoolExecutor
import time

cnxn_pool = cx.SessionPool('PVD_MART_30AUG','rxlogix','10.100.22.99:1521/PVSDEVDB',min=12,max=15,encoding='UTF-8')

def include_keys(dic, keys):
    key_set = set(keys) & set(dic.keys())
    return {key: dic[key] for key in key_set}

def pre_proc(data):
        print(type(data))
        for k,v in data.items():
            if isinstance(v,dict):
                if 'ArrayElem' in v.keys():
                # for k1,v1 in v.items():
                    data[k]=v['ArrayElem']
        return data
    
def flatten_list(d):
    try:
        key, lst = next((k, v) for k, v in d.items() if isinstance(v, list))
    except (StopIteration, AttributeError):
        return [flatten(d,'.')]
    return [flatten({**d, **{key: v}},'.') for record in lst for v in flatten_list(record)]

def generate_inserts(table, data):
    jsons = flatten_list(data)
    statements = []
    ins = jsons[0]
    ins = f"INSERT INTO {table} ({','.join(ins.keys())}) VALUES ({','.join([':'+str(v) for v in range(len(ins.keys()))])})"
    val_array = []
    for i in jsons:
        cols = i.keys()
        values = i.values()
        # ins = f"INSERT INTO {table} ({','.join(cols)}) VALUES ({','.join(['%s']*len(cols))})"
        val_array.append(tuple(values))
    statements.append(ins)
    statements.append(val_array)
    return statements

def return_all_inserts(dictionary)->dict:
    generic_keys = ['TENANT_ID','CASE_ID','VERSION_NUM']
    inserts = {}
    dictionary2 = pre_proc(dictionary)
    for i in dictionary2:
        if i not in generic_keys:
            t1 = include_keys(dictionary, generic_keys+[i])
            for j in t1:
                if isinstance(t1[j],dict):
                    if 'ArrayElem' in t1[j].keys():
                        t1[j] = t1[j]['ArrayElem']
            inserts[i] = tuple(generate_inserts(i,t1))
    return inserts


def push_data(dictionary1):
    print("Thread created")
    dictionary2 = return_all_inserts(dictionary1)
    cnxn = cnxn_pool.acquire()
    cursor = cnxn.cursor()
    cursor.execute("ALTER SESSION SET NLS_DATE_FORMAT = 'DD/MM/YYYY hh24:mi:ss'")
    print("""INCURSION :)(: STARTED""")
    counter = 1
    failed_insert = {}
    for key,val  in dictionary2.items(): 
        val_array = val[1]
        # print(val[0],val_array)
        try:
            cursor.executemany(val[0],val_array)
            # cnxn.commit()
        except cx.DatabaseError as e:
            print(e)
            failed_insert[key] = val
            # break
        # else:
            # print("Insert Done")
        # print(counter,key)
        counter+=1
    if len(failed_insert) == 0:
        cnxn.commit()
        print("All Inserts Committed")
    else:
        cnxn.rollback()
        print("Rollback Done")
        # print(failed_insert)
    print("data chunk inserted")
    cnxn_pool.release(cnxn)
    
        
# if __name__ == '__main__':
#     with open('jsons/msg1.json','r') as msg1:
#         string = msg1.read()
#         D = json.loads(string)
#         # dictionary = return_all_inserts(D)
#         push_data(D)
#     cnxn_pool.close()
    
def split_file(a,n):
    k,m = divmod(len(a),n)
    return list((a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n)))

def load(file)->list:
    with open(file,'r', encoding="utf8") as fp:
        string = fp.read()
        o = json.loads(string)
        line_chunk = split_file(o,1)
        print('chunk created')
        return line_chunk


if __name__ == "__main__":
    st = time.time()
    chunk_data = load('jsons/msg1.json')
    with ThreadPoolExecutor(max_workers=10) as threads:
        threads.map(push_data,chunk_data)
    cnxn_pool.close()
    et = time.time()
    print(et-st)

TRUNCATING THE TABLE

In [8]:
%%time
# ? trunacting the table
import json
import cx_Oracle as cx
cdns = cx.makedsn('10.100.22.99','1521',service_name='PVSDEVDB')
cnxn = cx.connect(user=r'PVD_MART_30AUG', password='rxlogix', dsn=cdns)
c = cnxn.cursor()
with open('read.json','r') as insert_stat:
        string = insert_stat.read()
        X = json.loads(string)
        arr = []
        for key,val  in X.items():
            arr.append(key,)
            c.execute(f"TRUNCATE TABLE {key}")
        print("truncate complete")
cnxn.close()
            

truncate complete
Wall time: 31.7 s


MULTI-THREADED DUMP

In [9]:
import json
import cx_Oracle as cx
from flatten_json import flatten
import time
from multiprocessing import Process,Pool
from  threading import  Thread
import boto3 as bt
import logging
#! Fetching sqs from dump msg
def recieve_message(url)->list:
    lst = []
    sqs_client = bt.client("sqs",region_name="ap-south-1")
    response = sqs_client.receive_message(
        QueueUrl = url,
        MaxNumberOfMessages=3,
        WaitTimeSeconds=0
    )
    val = response['Messages'][0]['Body']
    val2 = json.loads(val)
    bucket_name = val2['Records'][0]['s3']['bucket']['name']
    fileKey = val2['Records'][0]['s3']['object']['key']
    lst.append(bucket_name)
    lst.append(fileKey)
    return lst


################################################################################################

def include_keys(dic, keys):
    key_set = set(keys) & set(dic.keys())
    return {key: dic[key] for key in key_set}

#! Removing the ArrayElem
def pre_proc(data):
        for k,v in data.items():
            if isinstance(v,dict):
                if 'ArrayElem' in v.keys():
                    data[k]=v['ArrayElem']
        return data
    
#! Flattening the dictionary 
def flatten_list(d):
    try:
        key, lst = next((k, v) for k, v in d.items() if isinstance(v, list))
    except (StopIteration, AttributeError):
        return [flatten(d,'.')]
    return [flatten({**d, **{key: v}},'.') for record in lst for v in flatten_list(record)]

def generate_inserts(table, data):
    jsons = flatten_list(data)
    statements = []
    ins = jsons[0]
    ins = f"INSERT INTO {table} ({','.join(ins.keys())}) VALUES ({','.join([':'+str(v) for v in range(len(ins.keys()))])})"
    val_array = []
    for i in jsons:
        cols = i.keys()
        values = i.values()
        val_array.append(tuple(values))
    statements.append(ins)
    statements.append(val_array)
    return statements


#! Creating insert statements
def return_all_inserts(dictionary)->dict:
    generic_keys = ['TENANT_ID','CASE_ID','VERSION_NUM']
    inserts = {}
    dictionary2 = pre_proc(dictionary)
    for i in dictionary2:
        if i not in generic_keys:
            t1 = include_keys(dictionary, generic_keys+[i])
            for j in t1:
                if isinstance(t1[j],dict):
                    if 'ArrayElem' in t1[j].keys():
                        t1[j] = t1[j]['ArrayElem']
            inserts[i] = tuple(generate_inserts(i,t1))
    return inserts


################################################################################################
def push_data(dictionary1, cnxn_pool):
    global failures
    dictionary2 = return_all_inserts(dictionary1)
    st1 = time.time()
    cnxn = cnxn_pool.acquire()
    cursor = cnxn.cursor()
    cursor.execute("ALTER SESSION SET NLS_DATE_FORMAT = 'DD/MM/YYYY hh24:mi:ss'")
    print("""INCURSION :)(: STARTED \n""")
    failed_insert = {}
    for key,val  in dictionary2.items(): 
        val_array = val[1]
        try:
            cursor.executemany(val[0],val_array)
        except cx.DatabaseError as e:
            # print(e)
            failed_insert[key] = val
            # break
        # else:
            # print("Insert Done")
    if len(failed_insert) == 0:
        cnxn.commit()
        print("All Inserts Committed")
    else:
        cnxn.rollback()
        failures.append(failed_insert)
        print("Rollback Done")
    cnxn_pool.release(cnxn)

################################################################################################
        
# if __name__ == '__main__':
#     with open('jsons/msg1.json','r') as msg1:
#         string = msg1.read()
#         D = json.loads(string)
#         # dictionary = return_all_inserts(D)
#         push_data(D)
#     cnxn_pool.close()
# ! Splitting the file into multiple chunk_data 
def split_file(a,n):
    k,m = divmod(len(a),n)
    return list((a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n)))

def load(file)->list:
    with open(file,'r', encoding="utf8") as fp:
        string = fp.read()
        o = json.loads(string)
        line_chunk = split_file(o,1)
        print('chunk created')
        return line_chunk


if __name__ == "__main__":
    st = time.time()
    arr = recieve_message("https://sqs.ap-south-1.amazonaws.com/884379823401/testQueue")
    s3 = bt.client('s3')
    s3.download_file(Bucket=arr[0],Key=arr[1],Filename='C:/DEV/PY_DEV/output.json')
    failures = []
    cnxn_pool = cx.SessionPool('PVD_MART_30AUG','rxlogix','10.100.22.99:1521/PVSDEVDB',min=12,max=15,encoding='UTF-8')
    chunk_data = load('jsons/msg1.json')
    threads = []
    for i in chunk_data[0]:
        thread = Thread(target=push_data, args=(i,cnxn_pool,),daemon=True)
        threads.append(thread)
        thread.start()
    for t in threads:
        t.join()
        
    # while True:
    #     if any([t.is_alive() for t in threads]):
    #         print("Threads running")
    #         time.sleep(10)
    #     else:
    #         break
    cnxn_pool.close()
    #? Uploading Failed Inserts to S3
    with open('jsons/failues.json', 'w') as F:
        J = json.dumps(failures, encoding = 'utf-8',indent =1)
        F.write(J)
        s3.upload_file(F,Bucket=arr[0])
    print(f" length of failures : {len(failures)}")
    

chunk created
<class 'dict'><class 'dict'>

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
INCURSION :)(: STARTED 
INCURSION :)(: STARTED 


INCURSION :)(: STARTED 

INCURSION :)(: STARTED 

INCURSION :)(: STARTED 

INCURSION :)(: STARTED 

INCURSION :)(: STARTED 

INCURSION :)(: STARTED 

INCURSION :)(: STARTED 

INCURSION :)(: STARTED 

Rollback Done
Rollback Done
Rollback DoneRollback DoneRollback Done


Rollback Done
Rollback Done
Rollback Done
Rollback Done
Rollback Done
 length of failures : 10


In [3]:
import json 
def split_file(a,n):
    k,m = divmod(len(a),n)
    return list((a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n)))

def load(file)->list:
    with open(file,'r', encoding="utf8") as fp:
        string = fp.read()
        o = json.loads(string)
        line_chunk = split_file(o,1)
        print('chunk created')
        return line_chunk
    
chunk_data = load('jsons/msg1.json')
print(len(chunk_data[0]))
print(chunk_data)

chunk created
10
[[{'TENANT_ID': 1, 'CASE_ID': 159568016, 'VERSION_NUM': 1, 'C_ACTIONS_ADDL': {'ArrayElem': [{'CMT_REC_NUM': 100740, 'SYSTEM_CREATED': None, 'DELETED_FLAG': 0, 'DATE_VERSION_END': '31/12/9999 12:00:00', 'DELETED': None, 'DATE_VERSION_START': '01/01/1000 12:00:00', 'SENT_IN_LTR': None}, {'CMT_REC_NUM': 100739, 'SYSTEM_CREATED': None, 'DELETED_FLAG': 0, 'DATE_VERSION_END': '31/12/9999 12:00:00', 'DELETED': None, 'DATE_VERSION_START': '01/01/1000 12:00:00', 'SENT_IN_LTR': None}]}, 'C_AE_IDENTIFICATION': {'ArrayElem': [{'AE_COUNTRY_ID': 167, 'CD_DATE_AE_START': None, 'FLAG_SERIOUS_INT_REQ': None, 'FLAG_SERIOUS_DEATH': 1, 'MDR_AE_SOC': 'Pregnancy, puerperium and perinatal conditions', 'AE_COUNTRY_DESC': 'PERU', 'TXT_DATE_AE_START': None, 'DATE_AE_END': None, 'MDR_AE_HLT': 'Stillbirth and foetal death', 'AE_NATIVE_LANG_ID': None, 'MDR_AE_SOC_CODE': '10036585', 'MDR_AE_HLGT_CODE': '10000211', 'MDR_AE_SYN_CODE': None, 'MDR_AE_PT_CODE': '10055690', 'MDR_AE_LLT_CODE': '10053173',